In [7]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import threading
from dash import Dash, dcc, html, Input, Output, jupyter_dash

%matplotlib inline

In [8]:
df = pd.read_csv('Final_projects_files/project_1_python.csv', index_col='index',sep=',', decimal='.', encoding='UTF-8')

df.tail(1)

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,hosp_patients,total_tests,...,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,population,median_age,gdp_per_capita,life_expectancy,latitude,longitude
index,,,,,,,,,,,,,,,,,,,,,
190612,ZWE,Africa,Zimbabwe,2022-06-18,254753.0,NaN,5533.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,15092171.0,19.6,1899.775,61.49,-20.0,30.0


In [19]:


third_dash_app = Dash(__name__)


third_dash_app.layout = html.Div([
    html.Div([
        html.H1('COVID-19 Tracker', style={"color": "#27AE60", "text-align": "center"})
    ]),                           
    html.H3("Choose top x countries:", style={"text-align": "center", "color": "#7DCEA0"}),
    
    
    dcc.Slider(5, 20, 5,
        value=5,
        id='my-slider',
        marks={i: str(i) for i in range(5, 21, 5)}  
    ),   
    html.Br(),
    
    html.Div(children=[
        dcc.Graph(id='first-graph', style={'display':'inline-block', 'width': '48%'}),
        dcc.Graph(id='second-graph', style={'display':'inline-block', 'width': '48%'})
    ])
])


@third_dash_app.callback(
    Output(component_id='first-graph', component_property='figure'),
    [Input(component_id='my-slider', component_property='value')]
)
def generate_vaccinations_graph(n):

    df_copy = df.copy()
    
    max_date = df_copy['date'].max()
    df_filtered = df_copy[df_copy['date'] == max_date].sort_values(by='total_vaccinations', ascending=False).head(n).fillna(0)
    
    fig = px.bar(
        data_frame=df_filtered,
        x='location',
        y='total_vaccinations',
        title=f'Number of vaccinations',
        labels={'total_vaccinations': 'Total vaccinations'},
        color_discrete_sequence =['olive']*len(df)
    )

    fig.update_layout(xaxis_title = "Country")

    return fig


@third_dash_app.callback(
    Output(component_id='second-graph', component_property='figure'),
    [Input(component_id='my-slider', component_property='value')]
)
def generate_vaccination_ratio_graph(n):
    df_copy = df.copy()
    
    max_date = df_copy['date'].max()
    df_filtered = df_copy[df_copy['date'] == max_date].fillna(0)
    df_filtered['vaccination_ratio'] = df_filtered['total_vaccinations'] / df_filtered['population']
    df_filtered = df_filtered.sort_values(by='vaccination_ratio', ascending=False).head(n)
    
    fig = px.bar(
        data_frame=df_filtered,
        x='location',
        y='vaccination_ratio',
        title=f'Vaccination ratio',
        labels={'vaccination_ratio': 'Vaccination ratio'},
        color_discrete_sequence =['forestgreen']*len(df)
    )

    fig.update_layout(xaxis_title = "Country")
    return fig


if __name__ == '__main__':
    third_dash_app.run_server(mode='inline', port=8056)